In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
import xgboost as xgb

from sklearn.metrics import classification_report, accuracy_score 
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.metrics import confusion_matrix

In [4]:
#importing total X_train(features)
X = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X.csv')

In [5]:
#importing total Y_train(target)
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/y.csv')

In [6]:

#model building and fitting
xg_clf = xgb.XGBClassifier(objective ='binary:logistic')
xg_clf.fit(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier()

In [7]:
#train accuracy
xg_clf.score(X, y)

0.8922013888888889

In [8]:
yPred = xg_clf.predict(X)

In [9]:
#various performance scores
acc = accuracy_score(y, yPred)
print("The accuracy is {}".format(acc))
  
prec = precision_score(y, yPred)
print("The precision is {}".format(prec))
  
rec = recall_score(y, yPred)
print("The recall is {}".format(rec))
  
f1 = f1_score(y, yPred)
print("The F1-Score is {}".format(f1))
  
MCC = matthews_corrcoef(y, yPred)
print("The Matthews correlation coefficient is{}".format(MCC))

The accuracy is 0.8922013888888889
The precision is 0.602527727624452
The recall is 0.0876
The F1-Score is 0.15296133404633802
The Matthews correlation coefficient is0.20036695935642138


In [10]:
#ks score function
def ks_(data=None,target=None, prob=None):
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 10)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100
    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    kstable.index = range(1,11)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    print(kstable)
    
    return 

In [11]:
#creating data frame to check ks score for total training data
ks=pd.DataFrame(xg_clf.predict_proba(X))
del ks[0]

In [12]:
ks['target']=y

#ks table
ks_(ks,'target',1)

        min_prob  max_prob   events  nonevents event_rate nonevent_rate  \
Decile                                                                    
1       0.210570  0.864961  27386.0    44614.0     34.23%         6.97%   
2       0.139759  0.210570  13435.0    58565.0     16.79%         9.15%   
3       0.107841  0.139757   9250.0    62750.0     11.56%         9.80%   
4       0.088646  0.107840   7147.0    64852.0      8.93%        10.13%   
5       0.076510  0.088646   5683.0    66318.0      7.10%        10.36%   
6       0.067599  0.076510   4819.0    67181.0      6.02%        10.50%   
7       0.060164  0.067598   4077.0    67900.0      5.10%        10.61%   
8       0.053285  0.060164   3451.0    68572.0      4.31%        10.71%   
9       0.045724  0.053285   2765.0    69235.0      3.46%        10.82%   
10      0.018012  0.045724   1987.0    70013.0      2.48%        10.94%   

       cum_eventrate cum_noneventrate    KS  
Decile                                       
1      

In [13]:
#creating data frame of cust ids with probabilities
X_t= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/X_t.csv')
test_cust= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_cust.csv')
result_model = pd.DataFrame(xg_clf.predict_proba(X_t))
result=pd.concat([test_cust['cust_id'],result_model[1] ], axis=1)
result.rename(columns={1:'target'},inplace=True)

In [14]:
result

,cust_id,target
0,10146566,0.092584
1,10203613,0.065898
2,10322129,0.031745
3,10373212,0.062743
4,10487539,0.049760
...,...,...
179995,45739460,0.039629
179996,56006782,0.036098
179997,61138340,0.047180
179998,65715392,0.035206


In [ ]:
#uploading results(probabilities)
result.to_csv('/content/drive/MyDrive/Colab Notebooks/results.csv',index=False)